<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Import Libraries
</h1>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import HTML,display
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,TensorDataset

import os
for dirname,_,filenames in os.walk('/kaggle/input/multiclass-hand-gesture-recognition/'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Data Loading
</h1>

In [ ]:
df=pd.read_csv('/kaggle/input/multiclass-hand-gesture-recognition/MultiClass_HandGestureRecognition_dataset.csv')

In [ ]:
df.shape

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Data Preprocessing
</h1>

In [ ]:
df=df.sample(n=10000).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['label'].unique()

In [ ]:
df['label'].value_counts()

In [ ]:
df['label']=df['label'].astype(str)

In [ ]:
df['label'].unique()

In [ ]:
df['label'].value_counts()

In [ ]:
class_labels=df['label'].unique().tolist()
class_labels.sort()
print(f"Length of targets: {len(class_labels)}")
print(class_labels)

In [ ]:
def show_images(data):
    random_samples=[]
    sample_labels=[]
    for label in class_labels:
        label_df=data.loc[data['label']==label]
        sample_one=label_df.sample(n=1)
        sample=sample_one.iloc[:,:-1].values
        sample=sample.astype(np.float)
        sample=np.reshape(sample,(96,96))
        random_samples.append(sample)
        sample_labels.append(list(sample_one.iloc[:,-1:].values[0])[0])
    plt.figure(figsize=(15,15))
    for i in range(len(sample_labels)):
        ax=plt.subplot(7,5,i+1)
        plt.imshow(random_samples[i],cmap='gray')
        plt.axis('off')
        plt.title(label=f'class : {sample_labels[i]}',color='red')
show_images(df)

In [ ]:
df['label'].dtype

In [ ]:
class_dict={}
for idx,label in enumerate(class_labels):
    class_dict[label]=idx
print(class_dict)

In [ ]:
df['label']=df['label'].map(class_dict)
print(df['label'].unique().tolist())

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Data Splitting
</h1>

In [ ]:
X=df.iloc[:,:-1].values.astype(np.float)
X[:1]

In [ ]:
y=df.iloc[:,-1].values
y

In [ ]:
df_train,df_remaining=train_test_split(df,test_size=0.2,random_state=42,shuffle=True)

In [ ]:
df_train.shape,df_remaining.shape

In [ ]:
df_val,df_test=train_test_split(df_remaining,test_size=0.5,random_state=42,shuffle=True)
df_val.shape,df_test.shape

In [ ]:
class HGR_Dataset(Dataset):
    def __init__(self,data):
        super(HGR_Dataset,self).__init__()
        self.x=torch.from_numpy(data.iloc[:,:-1].values)
        self.x=self.x.reshape(self.x.size(0),1,96,96).squeeze(1)
        self.x=self.x.float()
        
        self.y=torch.from_numpy(data.iloc[:,-1].values)
        self.y=self.y.long()
        
        self.num_samples=data.shape[0]
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.num_samples

In [ ]:
train_dataset=HGR_Dataset(df_train)
val_dataset=HGR_Dataset(df_val)
test_dataset=HGR_Dataset(df_test)

In [ ]:
train_loader=DataLoader(dataset=train_dataset,
                       batch_size=50,
                       shuffle=True)
val_loader=DataLoader(dataset=val_dataset,
                     batch_size=50,
                     shuffle=True)
test_loader=DataLoader(dataset=test_dataset,
                      batch_size=50,
                      shuffle=True)

In [ ]:
print(f"train_loader:\n number of batches: {len(train_loader)}\n number of records: {len(train_loader.dataset)}")
print(f"val_loader:\n number of batches: {len(val_loader)}\n number of records: {len(val_loader.dataset)}")
print(f"val_loader:\n number of batches: {len(val_loader)}\n number of records: {len(test_loader.dataset)}")

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;text-align:center'>RNN model
</h1>

In [ ]:
class RNN_model(nn.Module):
    def __init__(self,input_size_,hidden_size_,num_layers_,num_classes,sequence_length):
        super(RNN_model,self).__init__()
        self.num_layers=num_layers_
        self.hidden_size=hidden_size_
        
        self.rnn=nn.RNN(input_size=input_size_,
                       hidden_size=hidden_size_,
                       num_layers=num_layers_,
                       batch_first=True)
        self.fc1=nn.Linear(in_features=sequence_length*hidden_size_,
                         out_features=1024)
        self.fc2=nn.Linear(in_features=1024,
                          out_features=num_classes)
    def forward(self,x):
        h0=torch.zeros(self.num_layers,x.shape[0],self.hidden_size).to(device) # h0 shape -> (2,50,256)
        rnn_output,hidden=self.rnn(x,h0) # rnn_output shape -> (50,96,256), hidden shape -> (2,50,256)
        rnn_output=rnn_output.reshape(rnn_output.shape[0],-1) # rnn_output shape -> (50,24576)
        fc1_output=self.fc1(rnn_output) # fc1_output shape -> (50,1024)
        fc2_output=self.fc2(fc1_output) # fc2_output shape -> (50,35)
        return fc2_output

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;text-align:center'>LSTM model
</h1>

In [ ]:
class LSTM_model(nn.Module):
    def __init__(self,input_size_,hidden_size_,num_layers_,num_classes,sequence_length):
        super(LSTM_model,self).__init__()
        self.num_layers=num_layers_
        self.hidden_size=hidden_size_
        
        self.lstm=nn.LSTM(input_size=input_size_,
                         hidden_size=hidden_size_,
                         num_layers=num_layers_,
                         batch_first=True)
        self.fc1=nn.Linear(in_features=sequence_length*hidden_size_,
                          out_features=1024)
        self.fc2=nn.Linear(in_features=1024,
                          out_features=num_classes)
    def forward(self,x):
        h0=torch.zeros(self.num_layers,x.shape[0],self.hidden_size).to(device) # h0 shape ->(2,50,256)
        c0=torch.zeros(self.num_layers,x.shape[0],self.hidden_size).to(device) # c0 shape ->(2,50,256)
        
        lstm_output, (h0,c0)=self.lstm(x,(h0,c0)) # lstm_output shape -> (50,96,256)
        lstm_output=lstm_output.reshape(lstm_output.shape[0],-1) # lstm_output -> (50,24576)
        fc1_output=self.fc1(lstm_output) # fc1_output -> (50,1024)
        fc2_output=self.fc2(fc1_output) # fc2_output -> (50,35)
        return fc2_output

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;text-align:center'>GRU model
</h1>

In [ ]:
class GRU_model(nn.Module):
    def __init__(self,input_size_,hidden_size_,num_layers_,num_classes,sequence_length):
        super(GRU_model,self).__init__()
        self.num_layers=num_layers_
        self.hidden_size=hidden_size_
        
        self.gru=nn.GRU(input_size=input_size_,
                       hidden_size=hidden_size_,
                       num_layers=num_layers_,
                       batch_first=True)
        self.fc1=nn.Linear(in_features=sequence_length*hidden_size_,
                          out_features=1024)
        self.fc2=nn.Linear(in_features=1024,
                          out_features=num_classes)
    def forward(self,x):
        h0=torch.zeros(self.num_layers,x.shape[0],self.hidden_size).to(device) # h0 shape -> (2,50,256)
        gru_output,hidden=self.gru(x,h0) # gru_output shape -> (50,96,256), hidden shape -> (2,50,256)
        gru_output=gru_output.reshape(gru_output.shape[0],-1) # gru_output shape -> (50,24576)
        fc1_output=self.fc1(gru_output) # fc1_output -> (50,1024)
        fc2_output=self.fc2(fc1_output) # fc2_output -> (50,35)
        return fc2_output

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Model Training
</h1>

In [ ]:
def train_loop(model,device,train_loader,val_loader,optimizer,criterion,batch_size,epochs,PATH):
    model=model.to(device)
    history={"train_accuracy":[],"train_loss":[],"val_accuracy":[],"val_loss":[]}
    min_loss=np.Inf
    for epoch in range(epochs):
        model.train()
        train_accuracy=0
        train_loss=0
        val_accuracy=0
        val_loss=0
        
        for X,y in train_loader:
            X=X.to(device)
            y=y.to(device)
            
            #forward propagation
            output=model(X)
            score,pred=output.max(1)
            loss=criterion(output,y)
            
            #backward propagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            cur_train_loss=loss.item()
            cur_train_accuracy=(pred==y).sum().item()/batch_size
            
            train_accuracy+=cur_train_accuracy
            train_loss+=cur_train_loss
        model.eval()
        with torch.no_grad():
            for X,y in val_loader:
                X=X.to(device)
                y=y.to(device)
                
                output=model(X)
                score,pred=output.max(1)
                loss=criterion(output,y)
                
                cur_val_loss=loss.item()
                cur_val_accuracy=(pred==y).sum().item()/batch_size
                
                val_loss+=cur_val_loss
                val_accuracy+=cur_val_accuracy
                
        train_accuracy=train_accuracy/len(train_loader)
        train_loss=train_loss/len(train_loader)
        val_accuracy=val_accuracy/len(val_loader)
        val_loss=val_loss/len(val_loader)
        
        print(f"[{epoch+1:>2d}/{epochs:>2d}], train_accuracy: {train_accuracy:>6f}, train_loss:{train_loss:>6f}, val_accuracy:{val_accuracy:>6f}, val_loss:{val_loss:>6f}")
        
        history['train_accuracy'].append(train_accuracy)
        history['train_loss'].append(train_loss)
        history['val_accuracy'].append(val_accuracy)
        history['val_loss'].append(val_loss)
        if val_loss<min_loss:
            torch.save(model.state_dict(),PATH)
            print(f"validation loss reduced from {min_loss:>2f} to {val_loss:>2f}")
        min_loss=val_loss
    return history

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Training RNN model
</h1>

In [ ]:
rnn_model=RNN_model(input_size_=96,
               hidden_size_=256,
               num_layers_=2,
               num_classes=35,
               sequence_length=96)

In [ ]:
rnn_optimizer=torch.optim.Adam(params=rnn_model.parameters(),lr=0.001)
rnn_criterion=nn.CrossEntropyLoss()

In [ ]:
rnn_history=train_loop(rnn_model,device,train_loader,val_loader,rnn_optimizer,rnn_criterion,batch_size=50,epochs=10,PATH='/kaggle/working/RecurrentNeuralNetwok_model.pt')

In [ ]:
with plt.style.context(style='fivethirtyeight'):
    plt.figure(figsize=(15,5))
    plt.rcParams['font.size']=15
    plt.subplot(121)
    plt.plot(rnn_history['train_accuracy'],label='train accuracy')
    plt.plot(rnn_history['val_accuracy'],label='val accuracy')
    plt.title(label='accuracy plots')
    plt.legend()
    plt.subplot(122)
    plt.plot(rnn_history['train_loss'],label='train loss')
    plt.plot(rnn_history['val_loss'],label='val loss')
    plt.title(label='loss plots')
    plt.legend()
    plt.show()

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Training LSTM model
</h1>

In [ ]:
lstm_model=LSTM_model(input_size_=96,
               hidden_size_=256,
               num_layers_=2,
               num_classes=35,
               sequence_length=96)

In [ ]:
lstm_optimizer=torch.optim.Adam(params=lstm_model.parameters(),lr=0.001)
lstm_criterion=nn.CrossEntropyLoss()

In [ ]:
lstm_history=train_loop(lstm_model,device,train_loader,val_loader,lstm_optimizer,lstm_criterion,batch_size=50,epochs=10,PATH='/kaggle/working/Long_short-term_memory_model.pt')

In [ ]:
with plt.style.context(style='fivethirtyeight'):
    plt.figure(figsize=(15,5))
    plt.rcParams['font.size']=15
    plt.subplot(121)
    plt.plot(lstm_history['train_accuracy'],label='train accuracy')
    plt.plot(lstm_history['val_accuracy'],label='val accuracy')
    plt.title(label='accuracy plots')
    plt.legend()
    plt.subplot(122)
    plt.plot(lstm_history['train_loss'],label='train loss')
    plt.plot(lstm_history['val_loss'],label='val loss')
    plt.title(label='loss plots')
    plt.legend()
    plt.show()

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Training GRU model
</h1>

In [ ]:
gru_model=GRU_model(input_size_=96,
               hidden_size_=256,
               num_layers_=2,
               num_classes=35,
               sequence_length=96)

In [ ]:
gru_optimizer=torch.optim.Adam(params=gru_model.parameters(),lr=0.001)
gru_criterion=nn.CrossEntropyLoss()

In [ ]:
gru_history=train_loop(gru_model,device,train_loader,val_loader,gru_optimizer,gru_criterion,batch_size=50,epochs=10,PATH='/kaggle/working/GatedRecurrentUnit_model.pt')

In [ ]:
with plt.style.context(style='fivethirtyeight'):
    plt.figure(figsize=(15,5))
    plt.rcParams['font.size']=15
    plt.subplot(121)
    plt.plot(gru_history['train_accuracy'],label='train accuracy')
    plt.plot(gru_history['val_accuracy'],label='val accuracy')
    plt.title(label='accuracy plots')
    plt.legend()
    plt.subplot(122)
    plt.plot(gru_history['train_loss'],label='train loss')
    plt.plot(gru_history['val_loss'],label='val loss')
    plt.title(label='loss plots')
    plt.legend()
    plt.show()

### Try it out

* 1)Due to memory error i have used only 10000 records from 42000. try to use all the records.
* 2)Use dropout layers to improve the performance of RNN and GRU models.
* 3)try inference with test loader
* 4)use some classification metrics like **classification report**, **confusion matrix** with the result of test data

<h1 style='background:black;color:white;font-family:tahoma;padding:10px;'>Thank You
</h1>